In [1]:
import pymongo
 
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

In [2]:
myclient.list_database_names()

['admin', 'config', 'local', 'tweet', 'tweet_project']

In [3]:
mydb = myclient["tweet_project"] # create a database test under the client

In [4]:
mydb.list_collection_names()

['origin_tweet', 'retweet_tweet', 'reply_tweet', 'quote_tweet']

In [5]:
mycol = mydb["origin_tweet"] 

In [6]:
myquery = { "reply_list" : { "$exists": "false"}}
mydoc = col_reply.find(myquery).limit(3).sort('timestamp_ms', -1)

NameError: name 'col_reply' is not defined

In [32]:
for x in mydoc:
    print(x)

{'_id': ObjectId('644261f72b4f18030a64063b'), 'tweet_id': '1253932995905564673', 'text': "@geertwilderspvv @WBHoekstra Waneer vertelt #PVV eindelijk eens de WAARHEID: die hele #lockdown is gebaseerd op foute, allang aangepaste, @WHO-cijfers: van 3,4% 'corona'doden naar 0,2%. Niks bijzonders, tóch lockdown? Waanzin! Massa-psychose. #HELP\n\nBeter ten halve gekeerd, ... 🇳🇱\n\nhttps://t.co/G65nHPMtlP", 'hashtags': [{'text': 'PVV', 'indices': [44, 48]}, {'text': 'lockdown', 'indices': [86, 95]}, {'text': 'HELP', 'indices': [243, 248]}], 'userid': '1026753348744237057', 'username': 'Sven Groter_3', 'user_screen_name': '3Groter', 'quote_count': 1, 'reply_count': 6, 'retweet_count': 13, 'favorite_count': 34, 'timestamp_ms': '1587826107842', 'reply_to_tweeet_id': '1253927939713966086', 'reply_to_user_id': '41778159', 'reply_to_user_name': 'geertwilderspvv', 'reply_list': ['1254028189220945920'], 'retweeted_list': ['1254059705883860992', '1254057370428284928', '1254033842299699201']}
{'_id': Ob

In [26]:
myquery = { "tweet_id" : { "$eq": "1253927939713966086"}}
mydoc = mycol.find(myquery).limit(3).sort('timestamp_ms', -1)
for x in mydoc:
    print(x)

{'_id': ObjectId('644260ef2b4f18030a63f850'), 'tweet_id': '1253927939713966086', 'text': 'Het is prima om KLM te helpen maar doe dan ook wat voor de gewone Nederlander in deze moeilijke tijden minister  @WBHoekstra: verlaag de huren en de btw op boodschappen, of schaf het eigen risico in de zorg af. Help bedrijven én burgers! \n\n#CoronaCrisis #corona \n#Wilders #PVV', 'hashtags': [{'text': 'CoronaCrisis', 'indices': [240, 253]}, {'text': 'corona', 'indices': [254, 261]}, {'text': 'Wilders', 'indices': [263, 271]}, {'text': 'PVV', 'indices': [272, 276]}], 'userid': '41778159', 'username': 'Geert Wilders', 'user_screen_name': 'geertwilderspvv', 'quote_count': 28, 'reply_count': 120, 'retweet_count': 410, 'favorite_count': 1440, 'timestamp_ms': '1587826092395', 'retweeted_list': ['1254059641094504449', '1254058582292783110', '1254056926679367681', '1254056862514909187', '1254056793023614981', '1254056414282240007', '1254055374992412673', '1254053295318020096', '1254051967078486016', '125

In [7]:
col_reply = mydb["reply_tweet"]
col_quoted = mydb["quote_tweet"]
col_retweet = mydb["retweet_tweet"]

In [8]:
import timeit

In [95]:
#Creating indexes
# resp = mycol.create_index([("text", -1)])
# print("index response", resp)

# mycol.createIndex(({"text": 'text', "hashtag":'text'}))

# mycol.create_index([("tweet_id",-1),("text", -1)])
# each_tweet["hashtags"], each_tweet["username"]

# col_reply = mydb["reply_tweet"]
# col_quoted = mydb["quote_tweet"]
# col_retweet = mydb["retweet_tweet"]
# col_retweet.create_index([("text",-1), ("username", -1), ("tweet_id", -1), ("favorite_count", -1)])

# collection.create_index([('your field', 'text')])
# mycol.create_index([('hashtags','text')])
# mycol.create_index([('text','text'),('tweet_id',-1)])
col_reply.index_information()



{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'text_text_hashtags_text_tweet_id_-1': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1), ('tweet_id', -1)],
  'weights': SON([('hashtags', 1), ('text', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [103]:
mycol.drop_indexes()

In [140]:
mycol.update_many(
    {},
    [
        { $multiply: [ "$favorite_count", 0.4] }
        { $multiply: [ <expression1>, <expression2>, ... ] }
        { $multiply: [ <expression1>, <expression2>, ... ] }
        {"$set": {"custom_score": { "$add": ["$favorite_count", "$reply_count", "$retweet_count", "$quote_count"]}}}
    ]
)


In [138]:
# col_reply.drop_indexes()
x = 0
try:

    for i in len(x):
        print(x)
except:
    print("key error or length error")

key error or length error


In [96]:
# %%timeit pass #-n 4 -r 2    


def SearchAlgo(searchString, criteria, sortMethod = "timestamp_ms", numofEntries = 1, numofReplies= 100, numofQuotes = 100, numofRetweets = 100, minimumResult = 10):
    if criteria in ["keyword", "hashtags", "tweet_id"]:
        if criteria == "keyword":
            myquery = {"$text": {"$search": None}}
            myquery['$text']['$search'] = searchString
            #We can also use favorite_count for sortMethod
            mydoc = mycol.find(myquery).sort(sortMethod, 1).limit(numofEntries)
        if criteria == "hashtags":
            myquery = {"hashtags.text": {"$eq": None}}
            myquery['hashtags.text']['$eq'] = searchString
            mydoc = mycol.find(myquery).sort(sortMethod, -1).limit(numofEntries)
        if criteria == "tweet_id":
            myquery = {"tweet_id": {"$eq": None}}
            myquery['tweet_id']['$eq'] = searchString
            mydoc = mycol.find(myquery).sort(sortMethod, 1).limit(numofEntries)
            mydoc = mycol.find(myquery).sort(sortMethod, -1).limit(numofEntries)
    list_of_results_original = list(mydoc)
    for each_tweet in list_of_results_original:
        print("\n Orginal tweet is: \n", each_tweet)
        try:
            print("Replies to the above tweet are as follows")
            for each_reply in range(min(len(each_tweet["reply_list"]), numofReplies)):
                reply_rec_list = recursive_reply(each_tweet["reply_list"][each_reply], reply_rec_list = [])
                [print(i) for i in reply_rec_list]
        except:
            print("Key missing for replies or no replies needed")

        try:
            print("Quoted tweets to the above tweet are as follows")
            for each_quote in range(min(len(each_tweet["quoted_list"]), numofQuotes)):
                x = quote_tweet(each_tweet["quoted_list"][each_quote])
                print(x)
        except:
            print("Key missing or no quoted tweets needed")

        try:
            print("Retweeted tweets to the above tweet are as follows")
            for each_retweet in range(min(len(each_tweet["retweeted_list"]), numofRetweets)):
                x = retweet_tweet(each_tweet["retweeted_list"][each_retweet])
                print(x)
        except:
            print("Key missing for retweets or no retweets needed")
            
    if len(list_of_results_original) < minimumResult:
        keywordSear_reply(searchString, numofEntries = 10-len(list_of_results_original))


def recursive_reply(key, reply_rec_list):
    reply_rec_list.append(reply_tweet(key))
    myquery_rec_reply = {"tweet_id": {"$eq": None}}
    myquery_rec_reply['tweet_id']['$eq'] = key
    mydoc_rec_reply = list(col_reply.find(myquery_rec_reply))
    for j_len in range(len(mydoc_rec_reply)):
        if "reply_list" in mydoc_rec_reply[j_len].keys():
            temp = mydoc_rec_reply[j_len]["reply_list"]
            for each_reply in temp:
                recursive_reply(each_reply, reply_rec_list)
    return reply_rec_list

            
def reply_tweet(reply_id):
    myquery_reply = { "tweet_id" : {"$eq" : None }}
    myquery_reply['tweet_id']['$eq'] = reply_id
    mydoc_reply = col_reply.find(myquery_reply)
    sam = list(mydoc_reply)[0]
    return [sam["tweet_id"], sam["text"], sam["username"], sam["favorite_count"]]

def keywordSear_reply(searchStr, numofEntries):
    criteria = "keyword"
    sortMethod = "timestamp_ms"
    if criteria == "keyword":
        myquery = {"$text": {"$search": None}}
        myquery['$text']['$search'] = searchStr
            #We can also use favorite_count for sortMethod
        mydoc = col_reply.find(myquery).sort(sortMethod, -1).limit(numofEntries)
    z = list(mydoc)
    for each_tweet in z:
        print("\n Search in reply yields: \n", each_tweet)

        
def quote_tweet(quote_id):
    myquery_quote = { "tweet_id" : {"$eq" : None }}
    myquery_quote['tweet_id']['$eq'] = quote_id
    mydoc_quote = col_quoted.find(myquery_quote)
    sam = list(mydoc_quote)[0]
    return [sam["tweet_id"], sam["text"], sam["username"], sam["favorite_count"]]

def retweet_tweet(retweet_id):
    myquery_retweet = { "tweet_id" : {"$eq" : None }}
    myquery_retweet['tweet_id']['$eq'] = retweet_id
    mydoc_retweet = col_retweet.find(myquery_retweet)
    sam = list(mydoc_retweet)[0]
    return [sam["tweet_id"], sam["text"], sam["username"], sam["favorite_count"]]


# SearchAlgo[""]: Data set 
# mydoc = SearchAlgo("covid", "hashtags")
# ["tweet_id"], 
#             each_tweet["text"], each_tweet["hashtags"], each_tweet["username"],each_tweet["quote_count"], 
#             each_tweet["reply_count"], each_tweet["retweet_count"], each_tweet["favorite_count"]
# mydoc = keywordSear("covid", "keyword")


SearchAlgo("1253927939713966086","tweet_id", "timestamp_ms", 1)


 Orginal tweet is: 
 {'_id': ObjectId('644260ef2b4f18030a63f850'), 'tweet_id': '1253927939713966086', 'text': 'Het is prima om KLM te helpen maar doe dan ook wat voor de gewone Nederlander in deze moeilijke tijden minister  @WBHoekstra: verlaag de huren en de btw op boodschappen, of schaf het eigen risico in de zorg af. Help bedrijven én burgers! \n\n#CoronaCrisis #corona \n#Wilders #PVV', 'hashtags': [{'text': 'CoronaCrisis', 'indices': [240, 253]}, {'text': 'corona', 'indices': [254, 261]}, {'text': 'Wilders', 'indices': [263, 271]}, {'text': 'PVV', 'indices': [272, 276]}], 'userid': '41778159', 'username': 'Geert Wilders', 'user_screen_name': 'geertwilderspvv', 'quote_count': 28, 'reply_count': 120, 'retweet_count': 410, 'favorite_count': 1440, 'timestamp_ms': '1587826092395', 'retweeted_list': ['1254059641094504449', '1254058582292783110', '1254056926679367681', '1254056862514909187', '1254056793023614981', '1254056414282240007', '1254055374992412673', '1254053295318020096', '1254

['1254045245714595841', 'RT @geertwilderspvv: Het is prima om KLM te helpen maar doe dan ook wat voor de gewone Nederlander in deze moeilijke tijden minister  @WBHo…', 'Annelies R', 0]
['1254044023398039553', 'RT @geertwilderspvv: Het is prima om KLM te helpen maar doe dan ook wat voor de gewone Nederlander in deze moeilijke tijden minister  @WBHo…', 'Peter von Querfurt', 0]
['1254043021127159810', 'RT @geertwilderspvv: Het is prima om KLM te helpen maar doe dan ook wat voor de gewone Nederlander in deze moeilijke tijden minister  @WBHo…', 'Delilah Hesselink', 0]
['1254042395706081280', 'RT @geertwilderspvv: Het is prima om KLM te helpen maar doe dan ook wat voor de gewone Nederlander in deze moeilijke tijden minister  @WBHo…', '🌹🌺Sas🌺🌹', 0]
['1254041835368046593', 'RT @geertwilderspvv: Het is prima om KLM te helpen maar doe dan ook wat voor de gewone Nederlander in deze moeilijke tijden minister  @WBHo…', 'Elly Weers', 0]
['1254041145463758855', 'RT @geertwilderspvv: Het is prima om K

In [71]:
#Wrapper Functions:
##1. Search by Keyword
def keywordSearch(x):
    SearchAlgo(x, "keyword", numofEntries = 1, sortMethod = "timestamp_ms", numofReplies= 0, numofQuotes = 0, numofRetweets = 0, minimumResult=0)
# keywordSearch("corona")
def hashtagSearch(x):
    SearchAlgo(x, "hashtags", numofEntries = 10, sortMethod = "timestamp_ms", numofReplies= 0, numofQuotes = 0, numofRetweets = 0, minimumResult=0)
hashtagSearch("zombie")
# def reply_specific_search(x):
#     SearchAlgo(x, "keyword", numofEntries = 1, sortMethod = "timestamp_ms", numofReplies= 1000, numofQuotes = 100, numofRetweets = 100, minimumResult = 0)

# # reply_specific_search("leader")

def advanced_search(x):
    SearchAlgo(x, "keyword", numofEntries = 1, sortMethod = "timestamp_ms", numofReplies= 1000, numofQuotes = 100, numofRetweets = 100, minimumResult = 10)

advanced_search("quran")



 Orginal tweet is: 
 {'_id': ObjectId('64425b582b4f18030a63df72'), 'tweet_id': '1254055023379701760', 'text': 'Media is spreading corona more through their hidden agenda. \n@ImranKhanPTI should take action these media houses. \n#molanaTariqJameel\n#TributeToSainGMSyed\nQuran\n#24YearsOfPTIstruggle\n#24YearsOfStruggle\n#hamidmir #level4lockdown', 'hashtags': [{'text': 'molanaTariqJameel', 'indices': [115, 133]}, {'text': 'TributeToSainGMSyed', 'indices': [134, 154]}, {'text': '24YearsOfPTIstruggle', 'indices': [161, 182]}, {'text': '24YearsOfStruggle', 'indices': [183, 201]}, {'text': 'hamidmir', 'indices': [202, 211]}, {'text': 'level4lockdown', 'indices': [212, 227]}], 'userid': '239458972', 'username': 'Rizwan Ahmad', 'user_screen_name': 'rizwan_309g', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 1, 'favorite_count': 0, 'timestamp_ms': '1587825601440', 'retweeted_list': ['1254057581879988230'], 'custom_score': 1, 'score': 0.5294117647058824}
Replies to the above tweet are as

TypeError: SearchAlgo() missing 1 required positional argument: 'criteria'

In [101]:
# myquery = { "favorite_count" : { "$ne": "0"}}
# mydoc = col_reply.find(myquery).limit(5)

In [102]:
# for x in mydoc:
#     print(x)

{'_id': ObjectId('64425b022b4f18030a636eb2'), 'tweet_id': '1254022773598572544', 'text': '@VinceMcMahon @TripleH We hereby honor to announce that we have developed a brand new treatment method using UV rays / Laser Ray that might be used effectively for preventing all mankind from infectious virus.\n\n#covid #covid19 #Covid_19 #corona #vascular #endovascular https://t.co/YkVlYtJiGL', 'hashtags': [{'text': 'covid', 'indices': [211, 217]}, {'text': 'covid19', 'indices': [218, 226]}, {'text': 'Covid_19', 'indices': [227, 236]}, {'text': 'corona', 'indices': [237, 244]}, {'text': 'vascular', 'indices': [245, 254]}, {'text': 'endovascular', 'indices': [255, 268]}], 'userid': '1091660129894838272', 'username': 'Milli teknoloji', 'user_screen_name': 'milliteknoloj', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'timestamp_ms': '1587817302499', 'reply_to_tweeet_id': '1253768179517657088', 'reply_to_user_id': '1222639789', 'reply_to_user_name': 'VinceMcMahon'}
{'_

In [ ]:
%%time


def SearchAlgo(searchString, criteria, sortMethod = "timestamp_ms", numofEntries = 10, numofReplies= 10, numofQuotes = 10, numofRetweets = 10, minimumResult = 10):
    if criteria in ["keyword", "hashtags", "tweet_id"]:
        if criteria == "keyword":
            myquery = {"$text": {"$search": None}}
            myquery['$text']['$search'] = searchString
#             myquery = {"text": {"$regex": None}}
#             myquery['text']['$regex'] = f'.*{searchString}.*'
            #We can also use favorite_count for sortMethod
#        ).sort( { score: { $meta: "textScore" } } )
# { $meta: <metaDataKeyword> }


            mydoc = mycol.find(myquery, {'score' : {'$meta' : 'textScore' }}).sort([('score', {'$meta' : 'textScore' })]).limit(numofEntries)
        if criteria == "hashtags":
            myquery = {"hashtags.text": {"$eq": None}}
            myquery['hashtags.text']['$eq'] = searchString
            mydoc = mycol.find(myquery).sort(sortMethod, -1).limit(numofEntries)
        if criteria == "tweet_id":
            myquery = {"tweet_id": {"$eq": None}}
            myquery['tweet_id']['$eq'] = searchString
            mydoc = mycol.find(myquery).sort(sortMethod, 1).limit(numofEntries)
            mydoc = mycol.find(myquery).sort(sortMethod, -1).limit(numofEntries)
    list_of_results_original = list(mydoc)
    for each_tweet in list_of_results_original:
        print("\n Orginal tweet is: \n", each_tweet)
        try:
            print("Replies to the above tweet are as follows")
            for each_reply in range(min(len(each_tweet["reply_list"]), numofReplies)):
                reply_rec_list = recursive_reply(each_tweet["reply_list"][each_reply], reply_rec_list = [])
                [print(i) for i in reply_rec_list]
        except:
            print("No further replies needed")

        try:
            print("Quoted tweets to the above tweet are as follows")
            for each_quote in range(min(len(each_tweet["quoted_list"]), numofQuotes)):
                x = quote_tweet(each_tweet["quoted_list"][each_quote])
                print(x)
        except:
            print("No further quoted tweets needed")

        try:
            print("Retweeted tweets to the above tweet are as follows")
            for each_retweet in range(min(len(each_tweet["retweeted_list"]), numofRetweets)):
                x = retweet_tweet(each_tweet["retweeted_list"][each_retweet])
                print(x)
        except:
            print("No further retweets needed")
            
#     if len(list_of_results_original) < minimumResult:
#         SearchAlgo(searchString, numofEntries = 10-len(list_of_results_original))


def recursive_reply(key, reply_rec_list):
    reply_rec_list.append(reply_tweet(key))
    myquery_rec_reply = {"tweet_id": {"$eq": None}}
    myquery_rec_reply['tweet_id']['$eq'] = key
    mydoc_rec_reply = list(col_reply.find(myquery_rec_reply))
    for j_len in range(len(mydoc_rec_reply)):
        if "reply_list" in mydoc_rec_reply[j_len].keys():
            temp = mydoc_rec_reply[j_len]["reply_list"]
            for each_reply in temp:
                recursive_reply(each_reply, reply_rec_list)
    return reply_rec_list

            
def reply_tweet(reply_id):
    myquery_reply = { "tweet_id" : {"$eq" : None }}
    myquery_reply['tweet_id']['$eq'] = reply_id
    mydoc_reply = mycol.find(myquery_reply)
    sam = list(mydoc_reply)[0]
    return [sam["tweet_id"], sam["text"], sam["username"], sam["favorite_count"]]

def keywordSear_reply(searchStr, numofEntries):
    criteria = "keyword"
    sortMethod = "timestamp_ms"
    if criteria == "keyword":
#         myquery = {"$text": {"$search": None}}
#         myquery['$text']['$search'] = searchStr
        myquery = {"text": {"$regex": None}}
        myquery['text']['$regex'] = f'.*{searchString}.*'
        #We can also use favorite_count for sortMethod
        mydoc = col_reply.find(myquery).sort(sortMethod, -1).limit(numofEntries)
    z = list(mydoc)
    for each_tweet in z:
        print("\n Search in reply yields: \n", each_tweet)
        
def top_10():
    print("Random top tweets ")
    mydoc = mycol.find().sort("custom_score": -1).limit(10)
#     $sort : { headline : 1 }
#     mydoc = mycol.find().sort("name")

#     for x in mydoc:
#         print(x)


    print(list(mydoc))
    
top_10()
        
def quote_tweet(quote_id):
    myquery_quote = { "tweet_id" : {"$eq" : None }}
    myquery_quote['tweet_id']['$eq'] = quote_id
    mydoc_quote = col_quoted.find(myquery_quote)
    sam = list(mydoc_quote)[0]
    return [sam["tweet_id"], sam["text"], sam["username"], sam["favorite_count"]]

def retweet_tweet(retweet_id):
    myquery_retweet = { "tweet_id" : {"$eq" : None }}
    myquery_retweet['tweet_id']['$eq'] = retweet_id
    mydoc_retweet = col_retweet.find(myquery_retweet)
    sam = list(mydoc_retweet)[0]
    return [sam["tweet_id"], sam["text"], sam["username"], sam["favorite_count"]]


# SearchAlgo("covid", "keyword")
# SearchAlgo("I refuse to work for the hospital because bitch y’all", "keyword")
# SearchAlgo("Tik tok", "keyword")
# SearchAlgo("\"Go Corona\"", "keyword")
# SearchAlgo("Covid 19", "keyword")


# SearchAlgo("1253927939713966086","tweet_id", "timestamp_ms", 1)
# SearchAlgo[""]: Data set 
# mydoc = SearchAlgo("covid", "hashtags")
# ["tweet_id"], 
#             each_tweet["text"], each_tweet["hashtags"], each_tweet["username"],each_tweet["quote_count"], 
#             each_tweet["reply_count"], each_tweet["retweet_count"], each_tweet["favorite_count"]

